In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings("ignore")

### Data Preprocessing

In [ ]:
# Load Orginal Data
sales_data = pd.read_excel("JANPO Sales.xlsx")
sales_data.drop(columns=['Unnamed: 0'], inplace = True)

# Check if Null
sales_data.info()

# Filter out rows with empty values
sales_data = sales_data[sales_data['QTY'] > 0]
sales_data = sales_data[sales_data['Amount'] > 0]
sales_data = sales_data[sales_data['Total'] > 0]

# Convert to dt format
sales_data['Date'] = pd.to_datetime(sales_data['Date'])

# Reformat
sales_data.pivot_table(index=['Date', 'Invoice#', 'Company', 'Country'], 
                       aggfunc=sum, 
                       values='Total').reset_index().sort_values(by=['Date'], ascending=False)

### Calculate the RFM Value, Create the RFM Table

In [ ]:
# Create the RFM table
NOW = dt.datetime(2020,8,28)

rfm_Table = sales_data.groupby(['Company', 'Country']).agg({'Date': lambda x: (NOW-x.max()).days,
                                                            'Invoice#': lambda x: len(x),
                                                            'Total': lambda x: sum(x)}).reset_index()

rfm_Table['Date'] = rfm_Table['Date'].astype(int)
rfm_Table.rename(columns={'Date': 'Recency', 
                          'Invoice#': 'Frequency', 
                          'Total': 'Monetary'}, inplace=True)

### Below 2 cells are optional

In [ ]:
# (Optional-1) Assign RFM Score
quantiles = rfm_Table.quantile(q=[0.25, 0.50, 0.75])
quantiles = quantiles.to_dict()

def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.750]:
        return 2
    else:
        return 1
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4

rfm_Table['R'] = rfm_Table['Recency'].apply(RScore, args=('Recency', quantiles))
rfm_Table['F'] = rfm_Table['Frequency'].apply(FMScore, args=('Frequency',quantiles))
rfm_Table['M'] = rfm_Table['Monetary'].apply(FMScore, args=('Monetary',quantiles))

In [ ]:
# (Optional-2) Create RFMGroup, FRMScore, Assign Loyalty_Level

# Concatenate the Score
rfm_Table['RFMGroup'] = rfm_Table['R'].map(str) + rfm_Table['F'].map(str) + rfm_Table['M'].map(str)

# Sum of the Score
rfm_Table['RFMScore'] = rfm_Table[['R', 'F', 'M']].sum(axis=1)

# Bronze (Worst), Silver, Gold, Platinum (Best)
Loyalty_Level = ['Bronze', 'Silver', 'Gold', 'Platinum']
Score_cuts = pd.qcut(rfm_Table['RFMScore'], q=4, labels=Loyalty_Level)
rfm_Table['Loyalty_Level'] = Score_cuts.values

## K-Means Clustering 

Check the Data Distribution

In [ ]:
# Check the Data Distribution
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(10,10))
sns.distplot(rfm_Table['Recency'], ax=axes[0])
sns.distplot(rfm_Table['Frequency'], ax=axes[1])
sns.distplot(rfm_Table['Monetary'], ax=axes[2])

Log Transformation

In [ ]:
# Handle negative and zero values so as to handle infinite numbers during log transformation
def handle_neg_n_zero(num):
    if num <= 0:
        return 1
    else:
        return num
    
# Apply handle_neg_n_zero function to Recency and Monetary columns 
rfm_Table['Recency'] = [handle_neg_n_zero(x) for x in rfm_Table['Recency']]
rfm_Table['Monetary'] = [handle_neg_n_zero(x) for x in rfm_Table['Monetary']]

# Perform Log transformation to bring data into normal or near normal distribution
Log_Tfd_Data = rfm_Table[['Recency', 'Frequency', 'Monetary']].apply(np.log, axis = 1).round(3)

In [ ]:
# After log transformation, check the Data Distribution again
fig, axes = plt.subplots(ncols=1, nrows=3, figsize=(10,10))
sns.distplot(Log_Tfd_Data['Recency'], ax=axes[0])
sns.distplot(Log_Tfd_Data['Frequency'], ax=axes[1])
sns.distplot(Log_Tfd_Data['Monetary'], ax=axes[2])

Data Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

#Bring the data on same scale
scaleobj = StandardScaler()
Scaled_Data = scaleobj.fit_transform(Log_Tfd_Data)

#Transform it back to dataframe
Scaled_Data = pd.DataFrame(Scaled_Data, index = rfm_Table.index, columns = Log_Tfd_Data.columns)

Elbow Method - Find the Optimum K

In [ ]:
from sklearn.cluster import KMeans

sum_of_sq_dist = {}
for k in range(1,15):
    km = KMeans(n_clusters= k, init= 'k-means++', max_iter= 1000)
    km = km.fit(Scaled_Data)
    sum_of_sq_dist[k] = km.inertia_
    
#Plot the graph for the sum of square distance values and Number of Clusters
sns.pointplot(x = list(sum_of_sq_dist.keys()), y = list(sum_of_sq_dist.values()))
plt.xlabel('Number of Clusters(k)')
plt.ylabel('Sum of Square Distances')
plt.title('Elbow Method For Optimal k')
plt.show()

Perform K-Mean Clustering

In [ ]:
KMean_clust = KMeans(n_clusters=4, init='k-means++', max_iter=1000)
KMean_clust.fit(Scaled_Data)

rfm_Table['Cluster'] = KMean_clust.labels_
rfm_Table

## Summary Analysis - Mean Value

In [ ]:
col = ['Recency', 'Frequency', 'Monetary', 'Cluster']
rfm_Table[col].groupby(['Cluster']).agg({'Recency':lambda x:round(x.mean(),2),
                                         'Frequency':lambda x:round(x.mean(),2),
                                         'Monetary':lambda x:round(x.mean(),2),
                                         'Cluster':'size'})

#### 個人解釋 

第1類顧客：(忠誠)各項顯示都是最好的，最高價值的客戶，對於這種客戶要精心維護，如果有新品上市或者大促活動，這部分人群應該是我們優先服務的

第3類的顧客，很久都沒有買了(R很差)，買過次數蠻多的，但是消費金額也是第二高的，代表買過很多次，但後來就不買了，應該是被挖角了。

第2類的顧客，最近剛買，頻率也低，但是消費金額也不高，應該是新客戶，應該設置相應的利益點，促進第二次購買。

第0類的顧客，很久都沒有買了(R最差)，頻率也最低，買了Sample後就不買了，是價格不好嗎？有什麼原因？

## Scatter Plot Frequency vs. Recency

In [ ]:
from matplotlib import pyplot as plt

# Scatter Plot Frequency vs. Recency
Colors = ["red", "green", "blue", 'black']
rfm_Table['Color'] = rfm_Table['Cluster'].apply(lambda p: Colors[p])
ax = rfm_Table.plot(kind='scatter', 
                    x='Recency', 
                    y='Frequency', 
                    figsize=(10,8), 
                    c=rfm_Table['Color'])